# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from pmdarima.arima import OCSBTest
from arch import arch_model
import seaborn as sns
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")
sns.set()


# Importing the Data

In [2]:
raw_data = yf.download(tickers='VOW3.DE,PAH3.DE, BMW.DE', interval="1d", group_by='ticker', auto_adjust=True, period='max')

[*********************100%***********************]  3 of 3 completed


In [3]:
# removing all nan values
raw_data = raw_data.dropna()

In [4]:
raw_data 

Ticker        PAH3.DE                                                VOW3.DE  \
Price            Open       High        Low      Close     Volume       Open   
Date                                                                           
2009-01-28  22.561689  23.815676  22.284704  23.528618   901566.0  20.198659   
2009-01-29  23.503435  23.568906  22.335063  22.627155   629474.0  21.643037   
2009-01-30  22.420677  24.122875  22.360244  23.166019  1467863.0  21.891286   
2009-02-02  22.797469  23.050192  21.955070  22.239380  1350452.0  22.004127   
2009-02-03  22.328884  22.476304  21.618108  22.144608  1183009.0  20.689520   
...               ...        ...        ...        ...        ...        ...   
2025-08-04  34.720001  34.900002  34.200001  34.240002   404823.0  89.620003   
2025-08-05  34.380001  34.939999  34.270000  34.750000   352825.0  89.099998   
2025-08-06  34.950001  35.349998  34.860001  34.990002   445415.0  90.739998   
2025-08-07  35.180000  35.790001  34.810001  35.209999   512989.0  91.279999   
2025-08-08  35.470001  36.080002  35.439999  35.889999   463596.0  93.760002   

Ticker                                                     BMW.DE             \
Price            High        Low      Close     Volume       Open       High   
Date                                                                           
2009-01-28  21.880001  20.114029  21.643034   268595.0   9.453350   9.711350   
2009-01-29  21.908215  20.717734  21.908215   175749.0   9.463368   9.563563   
2009-01-30  22.624758  21.439920  21.863075   330777.0   9.145251   9.470884   
2009-02-02  22.004127  19.578032  20.446913   217917.0   9.172803   9.257968   
2009-02-03  20.813646  18.985612  20.266365   210250.0   8.842161   9.095152   
...               ...        ...        ...        ...        ...        ...   
2025-08-04  89.779999  88.260002  88.680000   835263.0  82.040001  82.580002   
2025-08-05  90.720001  88.760002  90.300003   850993.0  83.040001  83.860001   
2025-08-06  92.019997  90.540001  91.059998   716169.0  84.440002  85.320000   
2025-08-07  93.940002  90.480003  93.260002   962648.0  84.940002  86.900002   
2025-08-08  95.739998  93.599998  95.379997  1105317.0  87.000000  87.980003   

Ticker                                     
Price             Low      Close   Volume  
Date                                       
2009-01-28   9.388223   9.508456  5214547  
2009-01-29   8.957386   9.167794  5951883  
2009-01-30   9.047562   9.323097  3368505  
2009-02-02   8.714413   8.822123  4149233  
2009-02-03   8.686860   9.080123  3953543  
...               ...        ...      ...  
2025-08-04  81.599998  82.580002   866240  
2025-08-05  82.419998  83.739998   746331  
2025-08-06  83.980003  84.739998   669215  
2025-08-07  84.419998  86.360001   994829  
2025-08-08  86.239998  87.440002  1447610  

[4201 rows x 15 columns]

In [5]:
df = raw_data.copy()

# Defining Key Dates

In [33]:
# starting date

start_date = "2009-06-01"

# First official announcement of VW's intention to buy Porsche = 49.9% stake in Porsche

announcement_1 = "2009-12-09"

# Second official announcement of VW's intention to buy Porsche = 50.1% stake in Porsche = VW has 100% control of Porsche

announcement_2 = "2012-07-05"

# Ending date

end_date = "2014-01-01"

# Dieselgate scandal = VW's CEO resigns

dieselgate = "2015-09-20"


# Pre processing the Data

In [7]:
# extractin the closing prices of the stocks

df['vol'] = df['VOW3.DE'].Close
df['porsche'] = df['PAH3.DE'].Close
df['bmw'] = df['BMW.DE'].Close

# creating returns
df['ret_vol'] = df['vol'].pct_change(1).mul(100)
df['ret_porsche'] = df['porsche'].pct_change(1).mul(100)
df['ret_bmw'] = df['bmw'].pct_change(1).mul(100)

# creating squared returns
df['ret_vol_sq'] = df['ret_vol'] ** 2
df['ret_porsche_sq'] = df['ret_porsche'] ** 2
df['ret_bmw_sq'] = df['ret_bmw'] ** 2

# Extracting volume
df['volume_vol'] = df['VOW3.DE'].Volume
df['volume_porsche'] = df['PAH3.DE'].Volume
df['volume_bmw'] = df['BMW.DE'].Volume

In [8]:
df = df.asfreq('b')
df = df.fillna(method='ffill')

In [9]:
# removing extra data
del df['VOW3.DE']
del df['PAH3.DE']
del df['BMW.DE']

In [10]:
df

Ticker,vol,porsche,bmw,ret_vol,ret_porsche,ret_bmw,ret_vol_sq,ret_porsche_sq,ret_bmw_sq,volume_vol,volume_porsche,volume_bmw
Price,,,,,,,,,,,,
Date,,,,,,,,,,,,
2009-01-28,21.643034,23.528618,9.508456,NaN,NaN,NaN,NaN,NaN,NaN,268595.0,901566.0,5214547.0
2009-01-29,21.908215,22.627155,9.167794,1.225247,-3.831345,-3.582727,1.501230,14.679206,12.835931,175749.0,629474.0,5951883.0
2009-01-30,21.863075,23.166019,9.323097,-0.206038,2.381493,1.694006,0.042452,5.671509,2.869657,330777.0,1467863.0,3368505.0
2009-02-02,20.446913,22.239380,8.822123,-6.477417,-3.999995,-5.373479,41.956927,15.999958,28.874275,217917.0,1350452.0,4149233.0
2009-02-03,20.266365,22.144608,9.080123,-0.883007,-0.426146,2.924471,0.779702,0.181601,8.552528,210250.0,1183009.0,3953543.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04,88.680000,34.240002,82.580002,-0.538362,-1.240255,0.584657,0.289833,1.538233,0.341824,835263.0,404823.0,866240.0
2025-08-05,90.300003,34.750000,83.739998,1.826796,1.489481,1.404694,3.337184,2.218554,1.973164,850993.0,352825.0,746331.0


# Plotting the prices

In [11]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.loc[start_date:end_date].index, y=df.loc[start_date:end_date]['vol'],
    mode='lines', name='Volkswagen', line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=df.loc[start_date:end_date].index, y=df.loc[start_date:end_date]['porsche'],
    mode='lines', name='Porsche', line=dict(color='red')
))
fig.add_trace(go.Scatter(
    x=df.loc[start_date:end_date].index, y=df.loc[start_date:end_date]['bmw'],
    mode='lines', name='BMW', line=dict(color='green')
))

# Add vertical lines for announcements (without annotation_text)
fig.add_vline(x=pd.to_datetime(announcement_1), line_color='orange', line_dash='dash')
fig.add_vline(x=pd.to_datetime(announcement_2), line_color='purple', line_dash='dash')
fig.add_vline(x=pd.to_datetime(dieselgate), line_color='black', line_dash='dash')

# Add annotations for each event
fig.add_annotation(
    x=pd.to_datetime(announcement_1),
    y=df['vol'].max(),  # or another y-value for label placement
    text='Announcement 1',
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40
)
fig.add_annotation(
    x=pd.to_datetime(announcement_2),
    y=df['vol'].max(),
    text='Announcement 2',
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40
)
fig.add_annotation(
    x=pd.to_datetime(dieselgate),
    y=df['vol'].max(),
    text='Dieselgate',
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40
)

fig.update_layout(
    width=1000,
    height=600,
    xaxis=dict(
        range=[df.loc[start_date:end_date].index.min(), df.loc[start_date:end_date].index.max()]
    )
)

fig.show()

# Correlation

In [34]:
print("Correlation among manufactures from : "+ str(start_date) + " to " + str(end_date))
print("Volkswagen and Porsche correlation: \t" + str(df.loc[start_date:end_date]['vol'].corr(df.loc[start_date:end_date]['porsche'])))
print("Volkswagen and BMW correlation: \t" + str(df.loc[start_date:end_date]['vol'].corr(df.loc[start_date:end_date]['bmw'])))
print("Porsche and BMW correlation: \t\t" + str(df.loc[start_date:end_date]['porsche'].corr(df.loc[start_date:end_date]['bmw'])))

Correlation among manufactures from : 2009-06-01 to 2014-01-01
Volkswagen and Porsche correlation: 	0.8462939334890721
Volkswagen and BMW correlation: 	0.9782149393180035
Porsche and BMW correlation: 		0.8163818214900013


In [35]:
# for start date and annoouncment 1 as end date 
print("Correlation among manufactures from : "+ str(start_date) + " to " + str(announcement_1))
print("Volkswagen and Porsche correlation: \t" + str(df.loc[start_date:announcement_1]['vol'].corr(df.loc[start_date:announcement_1]['porsche'])))
print("Volkswagen and BMW correlation: \t" + str(df.loc[start_date:announcement_1]['vol'].corr(df.loc[start_date:announcement_1]['bmw'])))
print("Porsche and BMW correlation: \t\t" + str(df.loc[start_date:announcement_1]['porsche'].corr(df.loc[start_date:announcement_1]['bmw'])))

Correlation among manufactures from : 2009-06-01 to 2009-12-09
Volkswagen and Porsche correlation: 	0.7952721744377997
Volkswagen and BMW correlation: 	0.7930277828306189
Porsche and BMW correlation: 		0.700319544399973


* **The stock prices for these brands weren't too similar before the start of the buyout**



In [37]:
# print announcement 1 as start date and announcement 2 as end date 
print("Correlation among manufactures from : "+ str(announcement_1) + " to " + str(announcement_2))
print("Volkswagen and Porsche correlation: \t" + str(df.loc[announcement_1:announcement_2]['vol'].corr(df.loc[announcement_1:announcement_2]['porsche'])))
print("Volkswagen and BMW correlation: \t" + str(df.loc[announcement_1:announcement_2]['vol'].corr(df.loc[announcement_1:announcement_2]['bmw'])))
print("Porsche and BMW correlation: \t\t" + str(df.loc[announcement_1:announcement_2]['porsche'].corr(df.loc[announcement_1:announcement_2]['bmw'])))

Correlation among manufactures from : 2009-12-09 to 2012-07-05
Volkswagen and Porsche correlation: 	0.7434172959291865
Volkswagen and BMW correlation: 	0.9795877961943547
Porsche and BMW correlation: 		0.7051170511877222


Key Changes
Volkswagen–Porsche correlation dropped slightly (~0.80 → ~0.74), suggesting their price movements became less tightly linked after the buyout process began.

Volkswagen–BMW correlation jumped significantly (~0.79 → ~0.98), showing very strong co-movement post-announcement — possibly due to broader sector or market effects influencing both.

Porsche–BMW correlation stayed about the same (~0.70 → ~0.71), indicating no major change in their relationship.

In [32]:
# announcement 2 as start date and end date
print("Correlation among manufactures from : "+ str(announcement_2) + " to " + str(end_date))
print("Volkswagen and Porsche correlation: \t" + str(df.loc[announcement_2:end_date]['vol'].corr(df.loc[announcement_2:end_date]['porsche'])))
print("Volkswagen and BMW correlation: \t" + str(df.loc[announcement_2:end_date]['vol'].corr(df.loc[announcement_2:end_date]['bmw'])))
print("Porsche and BMW correlation: \t\t" + str(df.loc[announcement_2:end_date]['porsche'].corr(df.loc[announcement_2:end_date]['bmw'])))

Correlation among manufactures from : 2012-07-05 to 2014-01-01
Volkswagen and Porsche correlation: 	0.9409608675974013
Volkswagen and BMW correlation: 	0.9291131173284665
Porsche and BMW correlation: 		0.949720019890707



* In the post-2012 phase, all three pairs became extremely highly correlated (0.93–0.95 range).

* The biggest change is Porsche–BMW: from ~0.70 before 2012 to 0.95, suggesting that after VW and Porsche’s integration, their stock price movements aligned much more closely with BMW’s — possibly reflecting stronger sector-wide influences or similar market drivers.

* VW–Porsche also went from moderate correlation (0.74) in the early buyout to very strong (0.94) once integration effects took hold.

In [38]:
# end _date to df.index[-1]

print("Correlation among manufactures from : "+ str(end_date) + " to " + str(df.index[-1]))
print("Volkswagen and Porsche correlation: \t" + str(df.loc[end_date:df.index[-1]]['vol'].corr(df.loc[end_date:df.index[-1]]['porsche'])))
print("Volkswagen and BMW correlation: \t" + str(df.loc[end_date:df.index[-1]]['vol'].corr(df.loc[end_date:df.index[-1]]['bmw'])))
print("Porsche and BMW correlation: \t\t" + str(df.loc[end_date:df.index[-1]]['porsche'].corr(df.loc[end_date:df.index[-1]]['bmw'])))

Correlation among manufactures from : 2014-01-01 to 2025-08-08 00:00:00
Volkswagen and Porsche correlation: 	0.9138825840282839
Volkswagen and BMW correlation: 	0.08905921785288427
Porsche and BMW correlation: 		0.17448424719282876


* The higher corelation of the Vol and Prosche says that these two companies are representation as a single entity but the other two correlations reduced drastically.

* **But why did this actually happen?** Because after vol buyout of porsche, the other companies must be slighty correlated towards the single big entity. This is due to other events like **dieselgate scandal** which we will explore further.